In [ ]:
import copy
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM

class CustomStarcoder2ForCausalLM(AutoModelForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        # The model is initialized normally.
    
    def add_front_transformer_block(self, copy_weights: bool = True):
        """
        Inserts a new transformer block at the beginning of the transformer's block list.
        
        Args:
            copy_weights (bool): If True, the new block is initialized as a deep copy of the 
                                 current first block. Otherwise, it is freshly constructed.
        """
        # Verify that the model has the expected attribute.
        if not hasattr(self, "transformer") or not hasattr(self.transformer, "h"):
            raise AttributeError("The model does not have attribute 'transformer.h'. "
                                 "Please adjust the code to match your model's architecture.")
        
        # Retrieve the current first transformer block.
        original_first_block = self.transformer.h[0]
        
        # Create a new block.
        new_block = copy.deepcopy(original_first_block) if copy_weights else type(original_first_block)()
        
        # Insert the new block at index 0.
        self.transformer.h.insert(0, new_block)
        
        # Update the configuration to reflect the extra layer.
        self.config.num_hidden_layers += 1